In [1]:
! pip install -U langchain langchainhub langchain-community langchain-openai chromadb BeautifulSoup4 --quiet

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
# Load docs
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
import bs4

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        # 仅解析网页中特定的 HTML 类（class），以减少处理的数据量。
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)


data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain_community.embeddings import ModelScopeEmbeddings

model_id = "iic/nlp_gte_sentence-embedding_chinese-base"
embeddings = ModelScopeEmbeddings(model_id=model_id)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key="712873090557c357188b37720d819703.pu7vBZJUK7LbHzSa",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

2024-05-21 13:30:49,941 - modelscope - INFO - PyTorch version 2.1.2+cpu Found.
2024-05-21 13:30:49,943 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2024-05-21 13:30:49,944 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-05-21 13:30:49,944 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-05-21 13:30:49,996 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 aacbf9e8ebe525a5896d4c89570c0097 and a total number of 976 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-21 13:30:51,844 - modelscope - WARNING - Model revision not specified, use revision: v1.1.0
Downloading: 100%|██████████| 917/917 [0

In [7]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [8]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever()

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("什么是 Task Decomposition?")

# question = "What are the approaches to Task Decomposition?"
# result = qa_chain({"query": question})
# result["result"]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:962: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


'Task Decomposition 是一种将复杂任务分解成更小、更易于管理的子目标的方法，这有助于有效地处理复杂任务。通过这种方法，智能体可以规划步骤，进行自我批评和反思，从而在未来的行动中改进并提高最终成果的质量。在长期规划和任务分解方面存在的挑战包括：在漫长的历史中做计划，以及在遭遇意外错误时，有效地探索解决方案空间和调整计划等。'